# Plot salinity with velocity vectors
Combine the salinity color plot with velocity vectors

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
#
from schimpy import schism_mesh
# viz
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
from holoviews import opts,dim
hv.extension('bokeh')
# widgets
import panel as pn
pn.extension()

In [ ]:
# for advanced viz ops
import datashader
import holoviews.operation.datashader as hd
from holoviews.operation.datashader import datashade, rasterize
import warnings
warnings.filterwarnings('ignore')

## Open the data sets

In [ ]:
varname = 'salinity'

In [ ]:
dsv = xr.open_dataset('../tests/data/m1_hello_schism/outputs/out2d_1.nc')
dsv

In [ ]:
file=f'../tests/data/m1_hello_schism/outputs/{varname}_2.nc'
ds = xr.open_dataset(file)
ds

## Read mesh information

In [ ]:
smesh = schism_mesh.read_mesh('../tests/data/m1_hello_schism/hgrid.gr3')

In [ ]:
smesh.n_nodes(), smesh.n_edges(), smesh.n_elems()

In [ ]:
nodes = pd.DataFrame(smesh.nodes,columns=['x','y','z'])
nodes
nodes[varname] = ds[varname].values[0,:,0]

In [ ]:
trimesh = hv.TriMesh((smesh.elems, hv.Points(nodes, vdims=varname)))

In [ ]:
trimesh = trimesh.opts(
    opts.TriMesh(cmap='rainbow4', cnorm='eq_hist', colorbar=True, node_alpha=0, edge_alpha=0, edge_color=varname, filled=True, height=400, 
                 inspection_policy='edges', tools=['hover'], width=400))

In [ ]:
trimesh.nodes.data

## Calculate velocity vectors from depth averaged X and Y velocity vectors

In [ ]:
vmag = np.sqrt(dsv.depthAverageVelX**2+dsv.depthAverageVelY**2)
vangle = np.arctan2(dsv.depthAverageVelY,dsv.depthAverageVelX)
vel = xr.Dataset({'mag':vmag,'angle': vangle})
vel## Calculate velocity vectors from depth averaged X and Y velocity vectors

## Map of velocity vectors

In [ ]:
def velocity_field(time, vector_size=1):
    data = vel.isel(time=time)
    vf = hv.VectorField((vel.coords['SCHISM_hgrid_node_x'],vel.coords['SCHISM_hgrid_node_y'],data.angle, data.mag))
    vf.opts(opts.VectorField(pivot='tip', color=dim('Magnitude'), magnitude=dim('Magnitude').norm()*2000*vector_size, rescale_lengths=False))
    return vf.opts(width=600)

In [ ]:
velocity_field(70,1)

## DynamicMap of salinity colors

In [ ]:
def salinity_mesh(time, depth):
    trimesh.nodes.data[varname] = ds[varname].values[time, :, depth]
    return rasterize(trimesh, precompute=False, dynamic=True, 
                        aggregator=datashader.mean(varname)).opts(
            colorbar=True, cmap='rainbow4', width=600, height=300, tools=['hover'])

In [ ]:
salinity_mesh(0,0)

## Overlay the salinity mesh with the velocity field

In [ ]:
def update(time, depth):
    return salinity_mesh(time,depth).opts(width=1000, height=500)*velocity_field(time, 1)

## Add widgets and show in a separate dashboard

In [ ]:
import panel as pn
pn.extension()

In [ ]:
time_slider= pn.widgets.DiscreteSlider(name='Time',
                                       options=dict(zip(ds.time.astype('str').values,
                                                        range(len(ds.time)))))
depth_slider= pn.widgets.Select(name='nSCHISM_vgrid_layers',
                                options=dict(zip(ds.nSCHISM_vgrid_layers.astype('str').values, 
                                                 range(len(ds.nSCHISM_vgrid_layers)))))

In [ ]:
slider_row = pn.Row(time_slider, depth_slider)

In [ ]:
map_row = pn.Row(pn.bind(update, time=time_slider, depth=depth_slider))

In [ ]:
dash = pn.Column(slider_row, map_row).servable(f'HelloSCHISM: {varname}')

In [ ]:
dash.show()